Import some library code


*   `tensorflow` is the machine learning library we're using
*   `os` helps us interact with files and folders
*   `matplotlib` is for displaying charts and images
*   `numpy` helps us work with data to prepare it for `tensorflow` and review it afterwards


In [39]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np

This code gets the data you will use to train your model: pictures of cats and dogs.


In [40]:
import tensorflow_datasets as tfds
(raw_training, raw_validation, raw_testing), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)


This function takes an image and a label as inputs. The image is then converted into three sets of numbers representing the colours red, green, and blue for every pixel in the image. The combination of these colours can form any other colour. The red, green, and blue values are then converted from numbers between 0 and 255, to numbers between -1 and 1, as the model has been trained to work with values in that range. Finally, the image is resized based on the `IMAGE_SIZE` constant, to match the size the model was previously trained on. In this case, it's a 160 by 160 pixel square.


In [41]:
IMAGE_SIZE = 160

training_data = None

# Resize an image, and convert it into a form that tensorflow can read more easily
def prep_image(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  return image, label

training_data = raw_training.map(prep_image)
validation_data = raw_validation.map(prep_image)
testing_data = raw_testing.map(prep_image)

These are versions of the functions from the previous project, so you can use them to test your model.

In [42]:
def get_image_from_url(image_url):
  # If the temporary test_image.jpg file already exists,
  # delete it so a new one can be made.
  if os.path.exists('/root/.keras/datasets/test_image.jpg'):
    os.remove('/root/.keras/datasets/test_image.jpg')

  image_path = tf.keras.utils.get_file('test_image.jpg', origin=image_url)
  return image_path

def print_predictions(predictions):
    for (prediction, number) in zip(predictions[0], range(1, len(predictions[0])+1)):
      print('{}. {} {:.2f}%'.format(number, prediction[1], prediction[2]*100))

def predict_with_old_model(image_url):
  image_path = get_image_from_url(image_url)

  image = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))

  plt.figure()
  plt.imshow(image)

  image = tf.keras.preprocessing.image.img_to_array(image)
  image = np.expand_dims(image, axis=0)

  prediction_result = original_model.predict(image, batch_size=1)
  predictions = tf.keras.applications.imagenet_utils.decode_predictions(prediction_result, top=15)

  print_predictions(predictions)

def predict_image(image_url):
  image_path = get_image_from_url(image_url)

  image = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))

  plt.figure()
  plt.imshow(image)

  image = tf.keras.preprocessing.image.img_to_array(image)
  image = np.expand_dims(image, axis=0)

  prediction_result = model.predict(image, batch_size=1)
  labels = metadata.features['label'].names
  print(labels[prediction_result.argmin()])

Import and test the MobileNetV2 model that you will retrain.



In [43]:
IMAGE_SHAPE = (IMAGE_SIZE,IMAGE_SIZE, 3)
original_model= tf.keras.applications.MobileNetV2(input_shape = IMAGE_SHAPE, include_top = False)
original_model.trainable = False
#predict_with_old_model('https://dojo.soy/predict-dog')


In the cell below, split your images into training, validation, and testing data.

In [44]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 10000
training_batches = training_data.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation_data.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
testing_batches = testing_data.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Add the new layers to the model, to allow it to be retrained.

In [45]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2)
model = tf.keras.Sequential([
  original_model,
  global_average_layer,
  prediction_layer
])
BASE_LEARNING_RATE = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=BASE_LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160 (Functional)    │ (None, 5, 5, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_9           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 2)                   │           2,562 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Set up your training epochs and train the new layers of the model.

In [ ]:
TRAINING_EPOCHS = 10
with tf.device('/device:GPU:0'):
  history = model.fit(training_batches,
                      epochs=TRAINING_EPOCHS,
                      validation_data=validation_batches)

Epoch 1/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 56s 114ms/step - accuracy: 0.3580 - loss: 4.3223 - val_accuracy: 0.5129 - val_loss: 0.6200
Epoch 2/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 38s 82ms/step - accuracy: 0.5054 - loss: 0.5850 - val_accuracy: 0.5357 - val_loss: 0.4767
Epoch 3/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 40s 82ms/step - accuracy: 0.5202 - loss: 0.4747 - val_accuracy: 0.5649 - val_loss: 0.4588
Epoch 4/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.5991 - loss: 0.4540 - val_accuracy: 0.9273 - val_loss: 0.4392
Epoch 5/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 42s 82ms/step - accuracy: 0.9425 - loss: 0.4537 - val_accuracy: 0.9454 - val_loss: 0.4001
Epoch 6/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 39s 90ms/step - accuracy: 0.9562 - loss: 0.3901 - val_accuracy: 0.9523 - val_loss: 0.4250
Epoch 7/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 38s 83ms/step - accuracy: 0.9569 - loss: 0.3974 - val_accuracy: 0.9527 - val_loss: 0.4338
Epoch 8/10


Use the `predict_image` function to test your model.

In [ ]:
url_list = ['https://drive.google.com/uc?export=download&id=1e3HOZz3JVnRr9pUAd2TG8BIj9JwNM_EP',
'https://drive.google.com/uc?export=download&id=1kgeGefxxK1vOzX3jBtSHLixQO1R1YwL1',
'https://drive.google.com/uc?export=download&id=1SAunvm1Zc4CCUQ8kqEpAae6uYuiZHaMb',
'https://drive.google.com/uc?export=download&id=1H0OXGnxGJ7lWbZ6v6ygHTI1WnwXH9h5g',
'https://drive.google.com/uc?export=download&id=1QKthhiKkSdnjMkSl8NgDxrXDevsVrcbz']



for i in url_list:
  predict_image(i)
